# Milestone 2 (Ryan)

***



## Task 1 - Introduction to Markdown

## Here is my Multi-level Header!

**Here is my bold text!** 

*Here is my italic text!*

- Here is my unordered list
* Here is my unordered list
+ Here is my unordered list

1. Here is my ordered list

Here is my image!
![image.png](attachment:9c064f77-8831-4354-9238-00e9a339dad3.png)

> Here is my quoted text!

Here is my link
https://www.ubc.ca/


## Task 2 - Introduction to Git merges, branches and more

Completed in previous lab

## Task 3 - Method Chaining & Python Programs

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from scripts import project_functions

df = project_functions.load_and_process('..\..\data\data_raw\*.csv')

## Task 4 - Conduct an EDA on your Dataset

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import missingno
import warnings
from scripts import project_functions

data1 = project_functions.load_and_process('..\..\data\data_raw\*.csv')
data1
df = pd.DataFrame(data = data1)
df

In [ ]:
## This function displays the size of the dataset in the form (# of rows, # of columns)
print(df.shape)

## This function displays the names of each individual column
print(df.columns)

In [ ]:
## This function displays the number of unique results for each column
df.nunique(axis=0)

In [ ]:
## This function provides a quick summary of the data, including count, mean, standard deviation, min and max for each variable.
df.describe(datetime_is_numeric=True)

In [ ]:
## This function organizes the dataset in a manner to only include quantitative information
df_quantitative = df.copy().drop(['language', 'review', 'recommended', 'received_for_free', 'written_during_early_access', 'game_id'], axis=1).astype(float)
df_quantitative

In [ ]:
## The function organizes the dataset in a manner to only include qualitative information
df_qualitative = df.copy().drop(['timestamp_created', 'timestamp_updated', 'votes_up', 'votes_funny', 'steam_algorithm_score', 'comment_count', 'num_games_owned', 'num_reviews', 'playtime_forever', 'playtime_last_two_weeks', 'playtime_at_review', 'last_played'], axis=1)
df_qualitative

In [ ]:
## This functions provides some basic information about the quantitative data
df_quantitative.describe(datetime_is_numeric=True)

In [ ]:
## This function creates a scatterplot of two variables, in this case 'num_games_owned' & 'num_reviews'
plot = df_quantitative.plot(kind='scatter', x='num_games_owned', y='num_reviews')
plot

In [ ]:
## This function creates a histogram for the data, in this case 'num_games_owned' is used
df['num_games_owned'].astype(float).plot(kind='hist', bins=1500, figsize=(12,6), facecolor='grey', edgecolor='black').set_xlim([0,1500])

In [ ]:
## This function provides a boxplot of the information, in this case
df_quantitative.boxplot('num_games_owned')

In [ ]:
## This function provides a heatmap of the correlation between each variable
## A positive correlation coefficient means that as one increase the other increases as well
## A negative correlation coefficient means that as one increases the other will decrease
## A correlation coefficient of zero implies that there is no relationship between the variables
corr = df_quantitative.corr()
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))

In [ ]:
## These functions changed the form of time from epoch to standard date and 24 hour clock
df['timestamp_created'] = pd.to_datetime(df['timestamp_created'], unit='s')
df['timestamp_updated'] = pd.to_datetime(df['timestamp_updated'], unit='s')
df['last_played'] = pd.to_datetime(df['last_played'], unit='s')

In [ ]:
## Please note the following code was originally found at "https://gist.github.com/jiahao87/c97214065f996b76ab8fe4ca1964b2b5"


import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

        
def time_series_plot(df):
    """Given dataframe, generate times series plot of numeric data by daily, monthly and yearly frequency"""
    print("\nTo check time series of numeric data  by daily, monthly and yearly frequency")
    if len(df.select_dtypes(include='datetime64').columns)>0:
        for col in df.select_dtypes(include='datetime64').columns:
            for p in ['D', 'M', 'Y']:
                if p=='D':
                    print("Plotting daily data")
                elif p=='M':
                    print("Plotting monthly data")
                else:
                    print("Plotting yearly data")
                for col_num in df.select_dtypes(include=np.number).columns:
                    __ = df.copy()
                    __ = __.set_index(col)
                    __T = __.resample(p).sum()
                    ax = __T[[col_num]].plot()
                    ax.set_ylim(bottom=0)
                    ax.get_yaxis().set_major_formatter(
                    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
                    plt.show()

                    
def numeric_eda(df, hue=None):
    """Given dataframe, generate EDA of numeric data"""
    print("\nTo check: \nDistribution of numeric data")
    display(df.describe().T)
    columns = df.select_dtypes(include=np.number).columns
    figure = plt.figure(figsize=(20, 10))
    figure.add_subplot(1, len(columns), 1)
    for index, col in enumerate(columns):
        if index > 0:
            figure.add_subplot(1, len(columns), index + 1)
        sns.boxplot(y=col, data=df, boxprops={'facecolor': 'None'})
    figure.tight_layout()
    plt.show()
    
    if len(df.select_dtypes(include='category').columns) > 0:
        for col_num in df.select_dtypes(include=np.number).columns:
            for col in df.select_dtypes(include='category').columns:
                fig = sns.catplot(x=col, y=col_num, kind='violin', data=df, height=5, aspect=2)
                fig.set_xticklabels(rotation=90)
                plt.show()
    
    # Plot the pairwise joint distributions
    print("\nTo check pairwise joint distribution of numeric data")
    if hue==None:
        sns.pairplot(df.select_dtypes(include=np.number))
    else:
        sns.pairplot(df.select_dtypes(include=np.number).join(df[[hue]]), hue=hue)
    plt.show()


def top5(df):
    """Given dataframe, generate top 5 unique values for non-numeric data"""
    columns = df.select_dtypes(include=['object', 'category']).columns
    for col in columns:
        print("Top 5 unique values of " + col)
        print(df[col].value_counts().reset_index().rename(columns={"index": col, col: "Count"})[
              :min(5, len(df[col].value_counts()))])
        print(" ")
    
    
def categorical_eda(df, hue=None):
    """Given dataframe, generate EDA of categorical data"""
    print("\nTo check: \nUnique count of non-numeric data\n")
    print(df.select_dtypes(include=['object', 'category']).nunique())
    top5(df)
    # Plot count distribution of categorical data
    for col in df.select_dtypes(include='category').columns:
        fig = sns.catplot(x=col, kind="count", data=df, hue=hue)
        fig.set_xticklabels(rotation=90)
        plt.show()
    

def eda(df):
    """Given dataframe, generate exploratory data analysis"""
    # check that input is pandas dataframe
    if type(df) != pd.core.frame.DataFrame:
        raise TypeError("Only pandas dataframe is allowed as input")
        
    # replace field that's entirely space (or empty) with NaN
    df = df.replace(r'^\s*$', np.nan, regex=True)

    print("Preview of data:")
    display(df.head(3))

    print("\nTo check: \n (1) Total number of entries \n (2) Column types \n (3) Any null values\n")
    print(df.info())

    # generate preview of entries with null values
    if df.isnull().any(axis=None):
        print("\nPreview of data with null values:")
        display(df[df.isnull().any(axis=1)].head(3))
        missingno.matrix(df)
        plt.show()

    # generate count statistics of duplicate entries
    if len(df[df.duplicated()]) > 0:
        print("\n***Number of duplicated entries: ", len(df[df.duplicated()]))
        display(df[df.duplicated(keep=False)].sort_values(by=list(df.columns)).head())
    else:
        print("\nNo duplicated entries found")

    # EDA of categorical data
    categorical_eda(df)
    
    # EDA of numeric data
    numeric_eda(df)
        
    # Plot time series plot of numeric data
    time_series_plot(df)



In [ ]:
## Provides information on the categorical data
categorical_eda(df)

In [ ]:
## This function shows the count of the top 5 most common results for each variable
top5(df)

In [ ]:
hist = df_quantitative.hist(bins=100, figsize=(20,10))
hist

In [ ]:
sns.countplot(data = df,  x='num_games_owned')

In [ ]:
sns.pairplot(df_quantitative)

## Task 5 - Conduct your Analysis to help answer your research questions

### Is there any correlation between receiving a game for free and the positivity of review?

The following code explores the dataset by comparing paid users vs. free users. Paid users are defined as users who did not receive the game for free. Free users are defined as users who did receive the game for free. 

The following relationships are explored:
- Average Votes Up
- Average Steam Algorithm Score
- Average Votes Funny
- Average Review Length
- Average Playtime at Time of Review
- Average Total Playtime
- Average Playtime for the Last Two Weeks

The code below imports all the functions necessary for this analysis. The final line of code is used to remove an outlier in the votes_funny column which contained over 4 billion votes. 

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import missingno
import warnings
from scripts1 import project_functions

data1 = project_functions.load_and_process('..\..\data\data_raw\*.csv')
data1
df = pd.DataFrame(data = data1)

df = df[df['votes_funny'] <= '40000000'] 

The following code is used to group the dataset into two categories, paid users and free users. 

In [ ]:
free_users = df[df.received_for_free == "True"]
paid_users = df[df.received_for_free == "False"]


The following code is used to create variables for the free user data points as well as assign them values.

In [ ]:
free_users_average_votes_up = free_users['votes_up'].astype(float).mean()
free_users_average_steam_algorithm_score = free_users['steam_algorithm_score'].astype(float).mean()
free_users_average_votes_funny = free_users['votes_funny'].astype(float).mean()
free_users_review_length = free_users['review'].str.len().mean()
free_users_average_playtime_at_review = free_users['playtime_at_review'].astype(float).mean()
free_users_average_playtime_forever = free_users['playtime_forever'].astype(float).mean()
free_users_average_playtime_last_two_weeks = free_users['playtime_last_two_weeks'].astype(float).mean()

The following code is used to create variables for the paid user data points as well as assign them values.

In [ ]:
paid_users_average_votes_up = paid_users['votes_up'].astype(float).mean()
paid_users_average_steam_algorithm_score = paid_users['steam_algorithm_score'].astype(float).mean()
paid_users_average_votes_funny = paid_users['votes_funny'].astype(float).mean()
paid_users_review_length = paid_users['review'].str.len().mean()
paid_users_average_playtime_at_review = paid_users['playtime_at_review'].astype(float).mean()
paid_users_average_playtime_forever = paid_users['playtime_forever'].astype(float).mean()
paid_users_average_playtime_last_two_weeks = paid_users['playtime_last_two_weeks'].astype(float).mean()

The following code is used to generate a bar graph comparing the **average votes up** for free users and paid users.

*Based on the results of the bar graph, we can see that reviews left by paid users have higher number of votes up on average than free users.*

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
xvalues = ['Avg. Votes Up (Free User)', 'Avg. Votes Up (Paid User)']
yvalues = [free_users_average_votes_up, paid_users_average_votes_up]
ax.bar(xvalues, yvalues, color=['green', 'black'])
plt.title('Avg. Votes Up for Free Users & Paid Users')
plt.show()

The following code is used to generate a bar graph comparing the **average steam algorithm** score for free users and paid users.

*Based on the results of the bar graph we can see that reviews left by paid users have a higher steam algorithm score on average than free users.*

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
xvalues = ['Avg. Steam Algorithm Score (Free User)', 'Avg. Steam Algorithm Score (Paid User)']
yvalues = [free_users_average_steam_algorithm_score, paid_users_average_steam_algorithm_score]
plt.bar(xvalues, yvalues, color=['green','black'])
plt.title('Avg. Steam Algorithm Score for Free Users & Paid Users')
plt.show()

The following code is used to generate a bar graph comparing the **average votes funny** for free users and paid users.

*Based on the results of the bar graph we can see that reviews left by paid users have a higher number of votes funny on average than free users.*

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
xvalues = ['Avg. Votes Funny (Free User)', 'Avg. Votes Funny (Paid User)']
yvalues = [free_users_average_votes_funny, paid_users_average_votes_funny]
plt.bar(xvalues, yvalues, color=['green','black'])
plt.title('Avg. Votes Funny for Free Users & Paid Users')
plt.show()

The following code is used to generate a bar graph comparing the **average length of review** for free users and paid users.

*Based on the results of the bar graph we can see that reviews left by paid users have longer review on average than free users.*

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
xvalues = ['Avg. Review Length (Free User)', 'Avg. Review Length (Paid User)']
yvalues = [free_users_review_length, paid_users_review_length]
plt.bar(xvalues, yvalues, color=['green','black'])
plt.title('Avg. Length of Review for Free Users & Paid Users')
plt.show()

The following code is used to generate a bar graph comparing the **average playtime at review** for free users and paid users.

*Based on the results of the bar graph we can see that paid users have a slightly lower average playtime at review when compared against free users.*

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
xvalues = ['Avg. Playtime at Review (Free User)', 'Avg. Playtime at Review (Paid User)']
yvalues = [free_users_average_playtime_at_review, paid_users_average_playtime_at_review]
plt.bar(xvalues, yvalues, color=['green','black'])
plt.title('Avg. Playtime at Review for Free Users & Paid Users')
plt.show()

The following code is used to generate a bar graph comparing the **average total playtime** for free users and paid users.

*Based on the results of the bar graph we can see that paid users have a slightly lower total playtime than free users.*

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
xvalues = ['Avg. Total Playtime (Free User)', 'Avg. Total Playtime (Paid User)']
yvalues = [free_users_average_playtime_forever, paid_users_average_playtime_forever]
plt.bar(xvalues, yvalues, color=['green','black'])
plt.title('Avg. Total Playtime for Free Users & Paid Users')
plt.show()

The following code is used to generate a bar graph comparing the **average playtime for the last two weeks** for free users and paid users.

*Based on the results of the bar graph we can see that paid users have a significanly lower average total playtime the last two weeks than free users.*

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
xvalues = ['Avg. Playtime for Last Two Weeks (Free User)', 'Avg. Playtime for Last Two Weeks (Paid User)']
yvalues = [free_users_average_playtime_last_two_weeks, paid_users_average_playtime_last_two_weeks]
plt.bar(xvalues, yvalues, color=['green','black'])
plt.title('Avg. Playtime for Last Two Weeks for Free Users & Paid Users')
plt.show()

### Conclusion

Differences in the reviews between paid and free users is evident from the analysis of this experiment. We can see that reviews left by free users tend to have a slightly lower steam algorithm score and significantly lower votes up, votes funny and overall length of review than paid users. Paid users may be more inclined to leave a better review for the simple fact that paid users made the choice to purchase the game and are thus more emotionally invested in it than someone who had received it for free. The significant differences between the average votes up and length of review point to an overall difference in quality of review between the two groups of users. The final three bar charts show that the free users tend to have slightly higher average total playtime than paid user and a significantly higher average playtime for the last two weeks. One explanation for the difference in playtime for the last two weeks is that someone who has received the game for free may not be an avid gamer and thus does not have very many games to choose from. Someone who has paid for the game likely considers it to be a hobby worth spending money on and thus would be more likely to own a larger set of games, thus reducing the amount of time they spend on the reviewed game. Interestingly enough, there was not a very large difference in the amount of playtime at time of review (although free users had a slightly higher average playtime). This means that regardless of whether a user paid for or received a game for free, the amount of time spent playing a game before a review was submitted did not change significantly. 